# Python-Code paketieren

(vgl. [Packaging Python Projects](https://packaging.python.org/tutorials/packaging-projects/))

Wenn wir eine Python-Anwendung mit mehreren Modulen erstellt haben, möchten wir diese in ein Paket zusammenbinden, damit wir diese Funktionalität leicht auf anderen Systemen installieren können.

Ein Python-Paket hat üblicherweise folgende Struktur (Beispiel Kartenspiel-Applikation):
```
cardplay/
    cardplay/
        play.py
        tests/
            test_play.py
    setup.cfg
    setup.py
```
Im obersten Verzeichnis liegt die Wurzel des (Git-) Repositories. Es enthält ein Unterverzeichnis mit dem gleichen Namen sowie die Files *setup.cfg* und *setup.py*.<br>
Das gleichnamige Unterverzeichnis ist das eigentliche Python-Modul. In diesem Beispiel liegt der Code im File *play.py*.<br>
Das File *setup.cfg* enthält Setup-Konfiguration. Diese wird benötigt, damit das Paket gebaut werden kann.

**Beispiel:**

*setup.cfg*
```
[metadata]
name = cardplay_benno-luthiger
version = 0.0.1
description = Simple card play exercise
url = https://github.com/eth-its/Python-for-SysAdmins/
author = Benno Luthiger
author_email = python.sysadmins@id.ethz.ch
license = MIT

[options]
zip_safe = False
packages = find:
```
- **name** ist der *distribution name* der des Pakets. Er muss eindeutig sein. Dies kann z.B. erreicht werden, wenn ein beliebiger Name mit dem Usernamen ergänzt wird.
- **version** ist die Version des Pakets.
- **description** ist eine kurze, einzeilige Zusammenfassung des Pakets.
- **packages** ist eine Liste aller importierten Python-Pakete, welche im Paket aufgenommen werden sollen. Diese Liste kann entweder manuell oder mit *find:* automatisiert erzeugt werden.

*setup.py*
```
from setuptools import setup

if __name__ == '__main__':
    setup()
```

Ist die Paketstruktur mit Code und Setup-Konfiguration bereit, kann das Paket erstellt werden. Zu diesem Zweck muss im Wurzelverzeichnis des Pakets folgender Befehl aufgerufen werden.:
```
$ pip install .
```
Damit ist das Python-Paket lokal installiert. Dies kann mit `pip list` überprüft werden.

Mit folgenden Befehlen kann ein Archiv des Python-Pakets erzeugt werden:
```
$ pip install --upgrade setuptools wheel
$ python setup.py sdist bdist_wheel
```
Mit dem ersten Befehl wird sichergestellt, dass die aktuellsten Versionen von *setuptools* und *wheel* installiert sind. Mit dem zweiten Befehl wird das Paket in ein *tar*-File verpackt und im Unterverzeichnis *dist* abgelegt.<br> Dieses File kann nun beispielsweise auf den Python Paket-Index (*PyPI*) [https://pypi.org/](https://pypi.org/) geladen werden. Dieser Upload auf *PyPI* erfolgt am besten mit Hilfe von [twine](https://twine.readthedocs.io/en/latest/).

## Versionsnummern

Die Versionsnummer eines Pakets, welches auf *PyPI* geladen werden soll, muss sich von einem existierenden Paket (mit dem gleichen Namen) unterscheiden. Sinnvollerweies wird die Versionsnummer im Format gemäss (*Semantic Versioning*)[https://semver.org/] angegeben: MAJOR.MINOR.PATCH.

Vor einem wichtigen Release (z.B. *1.0.0*) müssen unter Umständen sog. Alpha- und Beta-Release veröffentlicht werden. Eine gültie Versionshistorie kann wie folgt aussehen: `0.9.99` -> `1.0.0-alpha.1` -> `1.0.0-alpha.2` -> `1.0.0-beta.1` -> `1.0.0`.

## Kommandozeile

Wird eine Funktionalität, welche mit Python implementier worden ist, häufig gebraucht, ist es umständlich, immer `python my_func.py` eingeben zu müssen. Einfacher wäre es, nur `my_func` einzugeben.

Das kann mit dem *options.entry_points* in *setup.cfg* erreicht werden:
```
[options.entry_points]
console_scripts =
    executable-name = cardplay=cardplay.play:main
```
Mit dem *entry_points* `console_scripts` wird der Setup so konfiguriert, dass ein Kommandozeilen-Programm mit dem Namen *cardplay* erzeugt wird. Wird dieses gestartet, so wird die `main`-Funktion im Modul *cardplay.play* ausgeführt.

Wird in diesem Beispiel das Paket mit `python setup.py develop` erzeugt, kann es von der Kommandozeile mit `cardplay` gestartet werden.  

Weiterführende Informationen auf [Setuptools Integration](https://click.palletsprojects.com/en/7.x/setuptools/) oder [Command Line Scripts](https://python-packaging.readthedocs.io/en/latest/command-line-scripts.html).